#### 1. Import pandas library

#### 2. Import mysql.connector


#### 2.2. Run the following line


In [ ]:
cnx = mysql.connector.connect(user='guest', password='relational', host='relational.fit.cvut.cz', database='stats', use_pure=True)

### Import users table:

#### 3. Initialize cursor (as seen in classes' Jupyter Notebook)

#### 4. Initialize query  (as seen in classes' Jupyter Notebook) 
#### You want the users table from stats database. (Hint: stats.users)

#### 5. Send query to database  (as seen in classes' Jupyter Notebook)

#### 6. Create a new dataframe (preferably called users) with the following columns names (as seen in classes' Jupyter Notebook)

In [ ]:
columns_users = ['userId', 'Reputation', 'CreationDate', 'DisplayName', 'LastAccessDate', 'WebsiteUrl', 'Location', 'AboutMe', 'Views', 'UpVotes', 'DownVotes', 'AccountId', 'Age', 'ProfileImageUrl']


#### 7. Fill the new users dataframe with the data you can get iterating through cursor (as seen in classes' Jupyter Notebook)

#### 8. Rename Id column to userId

### Import posts table:

#### 9. Initialize cursor (as seen in classes' Jupyter Notebook)

#### 10. Initialize query  (as seen in classes' Jupyter Notebook) 
#### You want the posts table from stats database. (Hint: no hint this time ;) )

#### 11. Send query to database  (as seen in classes' Jupyter Notebook)

#### 12. Create a new dataframe (preferably called posts) with the following columns names (as seen in classes' Jupyter Notebook)

In [ ]:
columns_posts = ['Id', 'PostTypeId', 'AcceptedAnswerId', 'CreaionDate', 'Score', 'ViewCount', 'Body', 'OwnerUserId', 'LasActivityDate', 'Title', 'Tags', 'AnswerCount', 'CommentCount', 'FavoriteCount', 'LastEditorUserId', 'LastEditDate', 'CommunityOwnedDate', 'ParentId', 'ClosedDate', 'OwnerDisplayName', 'LastEditorDisplayName']

#### 12. Fill the new posts dataframe with the data you can get iterating through cursor (as seen in classes' Jupyter Notebook)

#### 13. Rename Id column to postId and OwnerUserId to userId

#### 14. Define new dataframes for users and posts with the following selected columns:
    **users columns**: userId, Reputation,Views,UpVotes,DownVotes
    **posts columns**: postId, Score,userID,ViewCount,CommentCount

#### 15. Merge both dataframes, users and posts. 
You will need to make a [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) of posts and users dataframes.

#### 16. How many missing values do you have in your merged dataframe? On which columns?

#### 17. You will need to make something with missing values.  Will you clean or filling them? Explain. 
**Remember** to check the results of your code before passing to the next step

#### 18. Adjust the data types in order to avoid future issues. Which ones should be changed? 